# Download and Load MABe Mouse Behavior Detection Dataset

This notebook downloads the [MABe Mouse Behavior Detection](https://www.kaggle.com/competitions/MABe-mouse-behavior-detection) competition dataset using the Python `kaggle` module.

## Prerequisites
1. **Kaggle Account**: Ensure you have joined the competition on Kaggle to accept the rules.
2. **API Token**: You need a `kaggle.json` file (API Token) from your account settings.
   - Place it in `~/.kaggle/kaggle.json` (Linux/Mac) or `%USERPROFILE%\.kaggle\kaggle.json` (Windows).
   - Or set `KAGGLE_USERNAME` and `KAGGLE_KEY` environment variables.

## Manual Dataset Placement

If you have already downloaded the dataset manually, please create a directory named `data` in the same location as this notebook.
Then, place the **unzipped contents** of the competition dataset directly into this `data` directory.

For example, if you downloaded `MABe-mouse-behavior-detection.zip`, after unzipping it, you would copy the folders and files (e.g., `train`, `test`, `sample_submission.csv`) directly into `./data/`.

If you proceed this way, you can skip running the download cells (Cells 3 and 4) in this notebook.

In [1]:
# Install necessary libraries
!pip install kaggle pandas

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 9.1 MB/s  0:00:01eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.2 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 8.1 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 9.8 MB/s  0:00:00m

In [2]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Initialize Kaggle API
api = KaggleApi()
try:
    api.authenticate()
    print("Kaggle API authenticated successfully.")
except Exception as e:
    print(f"Authentication failed: {e}")
    print("Please ensure 'kaggle.json' is in the correct location or environment variables are set.")

Could not find kaggle.json. Make sure it's located in /Users/zihanghuang/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


NameError: name 'exit' is not defined

In [ ]:
# Configuration
COMPETITION_NAME = 'MABe-mouse-behavior-detection'
DATA_DIR = './data'

# Create data directory if it doesn't exist
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

print(f"Downloading files for competition '{COMPETITION_NAME}' to '{DATA_DIR}'...")

try:
    # Download and unzip files
    api.competition_download_files(COMPETITION_NAME, path=DATA_DIR, unzip=True)
    print("Download and extraction complete.")
except Exception as e:
    print(f"An error occurred during download: {e}")
    print("Ensure you have accepted the competition rules on the Kaggle website.")

In [ ]:
# Inspect the downloaded files
print("Downloaded files:")
for root, dirs, files in os.walk(DATA_DIR):
    for file in files:
        print(os.path.join(root, file))

In [ ]:
# Example: Load a CSV file (adjust the filename based on the output above)
import pandas as pd

# Common files in competitions: 'train.csv', 'test.csv', 'sample_submission.csv'
# Update this variable with a file found in the previous step
TARGET_FILE = f'{DATA_DIR}/sample_submission.csv' 

if os.path.exists(TARGET_FILE):
    print(f"\nLoading {TARGET_FILE}...")
    try:
        df = pd.read_csv(TARGET_FILE)
        display(df.head())
        print(f"Loaded dataframe with shape: {df.shape}")
    except Exception as e:
        print(f"Could not load file: {e}")
else:
    print(f"\nFile '{TARGET_FILE}' not found. Please check the file list above and update 'TARGET_FILE'.")